<a href="https://colab.research.google.com/github/passivebook/FixWordPressLinks/blob/main/Fix_WordPress_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1T5JJmlDwdaEkAAbpH2VOn-TCeAuW8OKl

# Setup

In [31]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from bs4 import element as bs4_element
from google.colab import drive
import csv
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Set Google Drive Working Directory

Put the path to the google drive folder after `/content/gdrive/My Drive/`



In [3]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Fix WordPress Links")

## Upload the input.txt file to Google Drive 
1. Create a `input.txt` file and paste the WordPress Post Code in it. 
2. Upload the input.txt file to path setup above. 

## Check files


Check if files `input.txt` and `affiliate_links.csv` exists in Drive by running the code below. If you don't find the files, set the Google Drive working directory in the previous step. 

In [4]:
! ls

 affiliate_links.csv	      index_hyperlinked.html   output.txt
'Fix WordPress Links.ipynb'   input.txt


## Code Settings

In [32]:
filename = 'input.txt'
domain = 'passivebook.com'
affiliate_link = 'passivebook.com/go'

## Read the file

In [33]:
f = open(filename)
content = f.read()
#parse HTML
soup = BeautifulSoup(content, 'html.parser')

# Add Affiliate Links

In [34]:
html_doc = """
<!-- wp:paragraph -->
<p>This is a existing link <a class="test" href="https://yahoo.com/">Yahoo</a> Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another Google Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another lowercase bing Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Active Campaign Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Bunny CDN Text</p>
<!-- /wp:paragraph -->


<!-- wp:paragraph -->
<p>This is another multi word Bunny Text</p>
<!-- /wp:paragraph -->
"""

# Uncomment for Testing
# soup = BeautifulSoup(html_doc, 'html.parser')


# Read & Sort the CSV File
hyperlinks = pd.read_csv('affiliate_links.csv', engine='python')
hyperlinks['anchor_length'] = hyperlinks['anchor_to_link'].str.len()
hyperlinks = hyperlinks.sort_values(['link_to_be_added', 'anchor_length'], ascending=[False,False]) 
del hyperlinks['anchor_length']
hyperlinks = dict(hyperlinks.values)


# read the CSV file with anchor text and hyperlinks
# with open('affiliate_links.csv', 'r') as csv_file:
#   reader = csv.reader(csv_file)
#   hyperlinks = dict(reader)


# from bs4 import element as bs4_element
be_navStr = bs4_element.NavigableString
 
hList = [
    (anchor_text.strip(), hyperlink.strip()) for
    anchor_text, hyperlink in hyperlinks.items()
    if anchor_text.strip() and hyperlink.strip() # no blanks
]
 
 
for txt, link in hList:
    navStrs = [
        d for d in soup.descendants if type(d) == be_navStr 
        # and f' {txt.lower()} ' in f' {d.text.strip().lower()} ' # same as
        and f' {txt.lower()} ' in f' {d.string.strip().lower()} ' # same as
        # and (' '+txt.lower()+' ') in (' '+d.text.strip().lower()+' ')
    ]
    for ns in navStrs: 
        # tLen, remStr = len(txt), f' {ns.get_text().strip()} '
        tLen, remStr = len(txt), f' {ns.string.strip()} '
        # tLen, remStr = len(txt), f' {ns.text.strip()} '
        if remStr[1:-1].lower() == txt.lower():
            # to skip if it's already a hyperlink
            if ns.parent.name == 'a': 
                ns.parent['href'] = link # comment if you dont want to replace/update link
                continue 
        # Skip creating nested hyperlinks inside existing hyperlinks       
        if ns.parent.name == 'a': 
          continue 
 
        while f' {txt.lower()} ' in remStr.lower():
            sInd = remStr.lower().find(f' {txt.lower()} ') + 1
 
            hlTag = soup.new_tag('a', href=link)
            hlTag.append(remStr[sInd:sInd + tLen])
 
            newCont = [remStr[:sInd].lstrip(), hlTag, ' ']
            for addn in newCont: ns.insert_before(addn)
 
            remStr = f' {remStr[sInd + tLen:].strip()} ' 
        ns.replace_with(remStr.strip())

# Uncomment for Testing 
# print(soup)

# Fix Links

In [35]:
html_doc = """
<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/go/trello"> Affiliate Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/calculators/blog-earning-calculator/" class="test">Internal Link </a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://google.com">External Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="#jumplink">Jump Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/go/TRELLO.md">Markdown</a></p>
<!-- /wp:paragraph -->


<!-- wp:paragraph -->
<p><a href="/go/Rank Math%20Pro">Link with space</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is a Google Text</p>
<!-- /wp:paragraph -->
"""

# Uncomment for Testing
# soup = BeautifulSoup(html_doc, 'html.parser')

In [36]:
# Operations on All Links
for link in soup.find_all('a'):
    # Remove .md Obsidian Markdown from links
    link['href'] = link['href'].replace(".md", "")
    # Check if link is not an anchor link
    if "#" not in link['href']:
      # Set All Links to Open in New Tab
      link['target'] = '_blank'
      # Remove Leading and Trailiing White Spaces
      link.string = link.string.strip()
    # Add Domain to Internal Links
    if str(link['href'][0:1]) == "/":
      link['href'] = "https://" + domain + link['href'] 

# Internal Links
for link in soup.find_all('a', href=re.compile(domain)):
    # Convert all internal links to lower case
    link['href'] = link['href'].lower()        
    url = link['href']
    # Add Trailing / to all links
    if url[len(url)-1] != "/" and "#" not in link['href']:
        link['href'] = url + "/"
    # Replace space in internal links with -    
    link['href'] = link['href'].replace(" ", "-") 
    link['href'] = link['href'].replace("%20", "-")   

# Affiliate Links
for link in soup.find_all('a', href=re.compile(affiliate_link)):
    # Make Affiliate Links No Follow
    link['rel'] = 'nofollow'

# External Links
#for link in soup.find_all('a', href=re.compile(".*?:\\/\\/(?!"+domain+").*?")):
    #link['rel'] = link['rel'].remove('nofollow')
    #link['rel'] = 'nofollow'

# Uncomment for Testing
# print(soup)

In [37]:
with open("output.txt", "w", encoding='utf-8') as file:
    file.write(str(soup))